## Q1

In [1]:
##1-1
file_padding=[[0,0,0,0,0,0,0,0],
              [0,2,0,2,1,3,2,0],
              [0,0,2,0,2,2,2,0],
              [0,1,0,1,3,1,1,0],
              [0,0,0,1,1,1,0,0],
              [0,0,1,3,4,1,0,0],
              [0,0,1,0,0,5,2,0],
              [0,0,0,0,0,0,0,0]]
##1-2 
file_filter=[[4,0,4,3,5,2],
             [0,5,3,5,4,5],
             [1,1,4,4,3,3],
             [1,4,5,3,4,1],
             [1,1,3,10,4,1],
             [0,1,1,3,9,3]]
##1-3
file_maxpooling=[[5,5,5],
                 [4,5,4],
                 [1,10,9]]

## Q2

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torch.nn as nn

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = dsets.MNIST(root ='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download = True)
                          
mnist_test = dsets.MNIST(root ='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download = True)

In [6]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [7]:
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 4, stride = 1, padding = 1),
            # ? x 32 x 27 x 27
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
            # ? x 32 x 13 x 13
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            # ? x 64 x 13 x 13
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
            # ? x 64 x 6 x 6
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 2, padding = 1),
            # ? x 128 x 3 x 3
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
            # ? x 128 x 2 x 2
        
        self.fc1 = torch.nn.Linear(128 * 2 * 2, 625, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = 1 - self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [8]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
print("Learning Finished!")

Learning started. It takes sometime.
[Epoch:    1] cost = 0.260818541
[Epoch:    2] cost = 0.0653349981
[Epoch:    3] cost = 0.0462695546
[Epoch:    4] cost = 0.0356797874
[Epoch:    5] cost = 0.0290069245
[Epoch:    6] cost = 0.0246454626
[Epoch:    7] cost = 0.0211415365


In [ ]:
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())